# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

# PlanLoopController — E2E Notebook (STATIC loop)

## Scope
We validate PlanLoopController behavior end-to-end in STATIC planning mode:
- REPLAN_REQUIRED loop iteration
- replan_ctx injection into EnginePlanner prompt
- bounded replanning (max_replans)
- bounded identical plan repeats (max_same_plan_repeats)
- HITL escalation when limits are exceeded

## Constraints (this session)
- We add **one notebook cell per message** (including its English markdown).
- We use **only Intergrax framework classes** (no fake adapters, no fake classes, no new custom structures).
- A single **Configuration cell** will contain all setup objects (runtime context/state, real LLM adapter, planners, controller, policy).
- Each test will be a **separate code cell**.


In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [2]:
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime_state import RuntimeState
from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime_context import RuntimeContext
from intergrax.runtime.drop_in_knowledge_mode.pipelines.contract import RuntimePipeline
from intergrax.runtime.drop_in_knowledge_mode.pipelines.pipeline_factory import PipelineFactory
from intergrax.runtime.drop_in_knowledge_mode.planning.engine_planner import EnginePlanner
from intergrax.runtime.drop_in_knowledge_mode.planning.plan_loop_controller import PlanLoopController, PlanLoopPolicy
from intergrax.runtime.drop_in_knowledge_mode.planning.step_executor import StepExecutor
from intergrax.runtime.drop_in_knowledge_mode.planning.step_planner import StepPlanner, StepPlannerConfig
from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest


config = RuntimeConfig(
    llm_adapter=LLMAdapterRegistry.create(LLMProvider.OLLAMA)
)


# ---------------------------------------------------------------------
# Runtime context + request + state
# ---------------------------------------------------------------------

ctx = RuntimeContext(config=config)

request = RuntimeRequest(
    session_id="planloop-test-session",
    user_id="planloop-test-user",
    message="Test message for PlanLoopController",
    attachments=[],
)

state = RuntimeState(
    context=ctx,
    request=request,
    run_id="planloop-e2e-run-001",
)
state.configure_llm_tracker()

# Capabilities ON — so the planner is not clamped
state.cap_websearch_available = True
state.cap_tools_available = True
state.cap_rag_available = True
state.cap_user_ltm_available = True

# ---------------------------------------------------------------------
# Planning + execution stack
# ---------------------------------------------------------------------

engine_planner = EnginePlanner(llm_adapter=config.llm_adapter)

step_planner = StepPlanner(StepPlannerConfig())

registry = RuntimePipeline.build_default_planning_step_registry()
step_executor = StepExecutor(registry=registry)

policy = PlanLoopPolicy(
    max_replans=3,
    max_same_plan_repeats=2,
)

plan_loop = PlanLoopController(
    engine_planner=engine_planner,
    step_planner=step_planner,
    step_executor=step_executor,
    policy=policy,
)

print("Configuration ready.")


Configuration ready.
